This notebook demonstrates a simple function-calling feature of a large language model (LLM) using a local model. We use [Ollama](https://github.com/ollama/ollama) to serve the local model. Additionally, we utilize LangChain's Tools calling feature, which is another term for function calling.

The example generates function-calling information based on the incoming question, similar to OpenAI's function-calling feature described [here](https://platform.openai.com/docs/guides/function-calling).

### Problem Statement
Let's assume we are developing a customer service chatbot that takes in a user’s phone number and checks if:
- The phone number is inactive and needs reactivation.
- It accepts the request to reactivate the phone number.

This is a simple, one-step use case for the agent, which could be part of a larger agentic pattern handling more complex use cases.

#### Install the packages

In [ ]:
from langchain.tools.render import render_text_description
from langchain_community.llms import Ollama
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter

#### Define Custom Tools/ Functions
Read about tools calling in langchain [here](https://blog.langchain.dev/tool-calling-with-langchain/)

In [ ]:
@tool
def phone_status_checker(phone: str):
    """Useful when you want to check the status of a phone number"""
    phone_statuses = {"1234567890": True, "0987654321": False}
    status = phone_statuses.get(phone, False)
    return {"active": status}


@tool
def phone_reactivator(phone: str):
    """Useful when you want to reactivate a phone number"""
    return {"message": "Your request for phone reactivation is submitted. Thanks"}

#### Define main class
The entire workflow is defined using the Langchain Expression Language [LCEL](https://python.langchain.com/v0.1/docs/expression_language/) and [Runnables](https://python.langchain.com/v0.1/docs/expression_language/interface/) pipe syntax, which greatly enhances code readability.

In [ ]:
class PhoneStatusAgent:
    def __init__(self):
        self.llm = Ollama(model="llama3", format="json")
        self.tools = [phone_status_checker, phone_reactivator]
        rendered_tools = render_text_description(self.tools)
        system_prompt = f"""You are an assistant that has access to the following set of tools.
            Here are the names and descriptions for each tool:

            {rendered_tools}
            Given the user input, return the name and input of the tool to use.
            Return your response as a JSON blob with 'name' and 'arguments' keys.
            The value associated with the 'arguments' key should be a dictionary of parameters."""
        self.prompt = ChatPromptTemplate.from_messages(
            [("system", system_prompt), ("user", "{input}")]
        )
        self.chat_history = []

    def run(self, input_text):
        question = input_text

				# Here first three chain components generate the correct function calling information.
				# While the final chain component takes the function_name and arguments information and invokes the actual function.
        chain = self.prompt | self.llm | JsonOutputParser() | self.tool_chain
        response = chain.invoke({"input": question})
        return response

    def tool_chain(self, model_output):
        tool_map = {tool.name: tool for tool in self.tools}
        chosen_tool = tool_map[model_output["name"]]
        return itemgetter("arguments") | chosen_tool

#### Execute the Agent

In [ ]:
pp = PhoneStatusAgent()
print(pp.run('Submit phone reactivate request: 0987654321'))
print(pp.run('Please reactivate my phone number: 1234567890'))

#### More on Langchain
LangChain has developed multiple interfaces _(`create_tool_calling_agent`)_ to support the function-calling feature for models that support it. See more details [here](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

- Rigorous testing and research is required to finalize the model for function calling when integrated with LangChain's interfaces.
- Look for models which natively support it like OpenAI's chatgpt4.0, mixtral. They are known to perform reasonably well for this feature.